# Process tactile image data
* processed tactile data saved into two arrays:
*        firing_SA and firing_RA of shape (n_samples, n_pins*n_pins)
* accompanied with third array giving the pin locations in the array
*        pins of shape (n_pins*2, 2) giving their locations
* n_pins is number of pins along side of square array (here 19)
* values of the SA and FA firing rates from module afferent_response.py
* overall, the processing is iterated as a list over the data sets 
*        speeds (10 sets)
*        textures (13 sets)

To run, first edit dir_data to path where data stored.

In [1]:
import os, json, pickle
import numpy as np
from afferent_response import AfferentResponse as ar

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [2]:
dir_data = os.environ["DATAPATH"] + r"/open/afferents-tactile-textures-jrsi2022"

speeds = ['10','20','30','40','50','60','70','80','90','100']
textures = ['0','0.5','1','1.5','2','2.5','3','3.5','4','4.5','5','5.5','6']

n_speeds = len(speeds)
n_textures = len(textures)

with open("tip_params/tipB", "r") as read_file:
    tip_params = json.load(read_file)

In [3]:
afferents = ar(False, **tip_params)
pins_init = afferents.init_pins(dir_data + rf"/0/10/throw_away.mp4")
print(f"n_pins={len(pins_init)}")

firing_SA, firing_RA, pins = [[] for _ in range(3)]
for i in range(n_textures):
    print(f'{i}/{n_textures}', end=' ')

    firing_SA_j, firing_RA_j, pins_j = [[] for _ in range(3)]
    for j in range(n_speeds):
        firing = afferents.firing(dir_data + rf"/{textures[i]}/{speeds[j]}/video.mp4", pins_init)
    
        firing_SA_j.append(np.stack([firing[0][f'{k}'] for k in range(len(firing[0]))]))
        firing_RA_j.append(np.stack([firing[1][f'{k}'] for k in range(len(firing[1]))]))
        pins_j.append(firing[2])
  
    firing_SA.append(firing_SA_j) 
    firing_RA.append(firing_RA_j) 
    pins.append(pins_j)

os.makedirs(dir_data + rf"/processed", exist_ok=True)

save_obj(firing_SA, dir_data + r"/processed/firing_SA")
save_obj(firing_RA, dir_data + r"/processed/firing_RA")
save_obj(pins, dir_data + r"/processed/pins")

n_pins=361
0/13 1/13 2/13 3/13 4/13 5/13 6/13 7/13 8/13 9/13 10/13 11/13 12/13 